In [1]:
import pandas as pd
import numpy as np

In [3]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [8]:
data_files = {"train":"train.csv","validation":"eval.csv","test":"test.csv"}

In [9]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [10]:
from datasets import load_dataset, load_metric
df = load_dataset("csv", data_files=data_files)
df

Using custom data configuration default-8aa9904f218d9ee4


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-8aa9904f218d9ee4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 21533
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5384
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 331
    })
})

In [11]:
df["train"][0]

{'label': 2,
 'text': 'Lidocaine and prilocaine cream application in children was studied in seven non-US studies (320 patients) and one US study (100 patients). In controlled studies, application of lidocaine and prilocaine cream for at least 1 hour with or without presurgical medication prior to needle insertion provided significantly more pain reduction than placebo. In children under the age of seven years, lidocaine and prilocaine cream was less effective than in older children or adults.'}

In [12]:
tokenized_df = df.map(preprocess_function, batched = True)

  0%|          | 0/22 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

In [16]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_df["train"],
    eval_dataset=tokenized_df["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)



PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [17]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 21533
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6730


Step,Training Loss
500,0.371100
1000,0.278400
1500,0.251800
2000,0.231400
2500,0.236900
3000,0.204000
3500,0.195500
4000,0.207000
4500,0.176900
5000,0.175100


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1500/special_toke

TrainOutput(global_step=6730, training_loss=0.21381798584075065, metrics={'train_runtime': 1770.198, 'train_samples_per_second': 60.821, 'train_steps_per_second': 3.802, 'total_flos': 4985389495887390.0, 'train_loss': 0.21381798584075065, 'epoch': 5.0})

In [18]:
predict_train = trainer.predict(tokenized_df["train"])
predictions_train, labels_train, _ = predict_train
predictions_train = np.argmax(predictions_train, axis=1)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 21533
  Batch size = 16


In [30]:
predict_val = trainer.predict(tokenized_df["validation"])
predictions_val, labels_val, _ = predict_val
predictions_val = np.argmax(predictions_val, axis=1)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5384
  Batch size = 16


In [31]:
predict_test = trainer.predict(tokenized_df["test"])
predictions_test, labels_test, _ = predict_test
predictions_test = np.argmax(predictions_test, axis=1)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 331
  Batch size = 16


In [36]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
def evaluate_metric(predictions, labels, data_type = ""):
  print(data_type + " Accuracy: ", accuracy_score(predictions, labels))
  print(data_type + " Classification Report")
  print(classification_report(labels, predictions, target_names=["Headings","Sub-Headings","Sections"]))

In [37]:
evaluate_metric(predictions_train, labels_train)

 Accuracy:  0.9395346677193145
 Classification Report
              precision    recall  f1-score   support

    Headings       0.96      0.95      0.95      1419
Sub-Headings       0.82      0.91      0.86      4304
    Sections       0.98      0.95      0.96     15810

    accuracy                           0.94     21533
   macro avg       0.92      0.94      0.93     21533
weighted avg       0.94      0.94      0.94     21533



In [38]:
evaluate_metric(predictions_val, labels_val)

 Accuracy:  0.8770430906389302
 Classification Report
              precision    recall  f1-score   support

    Headings       0.91      0.90      0.91       380
Sub-Headings       0.66      0.75      0.70      1015
    Sections       0.94      0.91      0.92      3989

    accuracy                           0.88      5384
   macro avg       0.84      0.85      0.84      5384
weighted avg       0.88      0.88      0.88      5384



In [39]:
evaluate_metric(predictions_test, labels_test)

 Accuracy:  0.9244712990936556
 Classification Report
              precision    recall  f1-score   support

    Headings       1.00      0.18      0.31        11
Sub-Headings       0.94      0.87      0.90        87
    Sections       0.92      0.98      0.95       233

    accuracy                           0.92       331
   macro avg       0.95      0.68      0.72       331
weighted avg       0.93      0.92      0.92       331

